In [ ]:
!pip install -U spacy -q

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
nlp = spacy.blank("en")
db = DocBin()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
f = open('/content/drive/MyDrive/Colab Notebooks/MTP after 15/Judge_extraction/judge_name_annotation.json')
TRAIN_DATA = json.load(f)

In [ ]:
TRAIN_DATA

{'classes': ['JUDGE'],
 'annotations': [['Conclusion:\r\n25) Admittedly, No-Dues Certificate was submitted by the contractee-Company on 21.09.2012 and\r\non their request Completion Certificate was issued by the appellant- Contractor. The contractee,\r\nafter a gap of one month, that is, on 24.10.2012, withdrew the No Dues Certificate on the grounds of\r\ncoercion and duress and the claim for losses incurred during execution of the Contract site was\r\nmade vide letter dated 12.01.2013, i.e., after a gap of 3 ½ (three and a half) months whereas the\r\nFinal Bill was settled on 10.10.2012. When the contractee accepted the final payment in full and\r\nfinal satisfaction of all its claims, there is no point in raising the claim for losses incurred during the\r\nexecution of the Contract at a belated stage which creates an iota of doubt as to why such claim was\r\n26) In view of the foregoing discussion, we set aside the judgment and order dated 12.01.2015\r\npassed by the High Court. The 

In [ ]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
           print("Skipping entity")
        else:
           ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data_j_extract.spacy")

100%|██████████| 30/30 [00:00<00:00, 522.29it/s]


In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

2022-10-05 12:18:14.838309: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data_j_extract.spacy --paths.dev ./training_data_j_extract.spacy

2022-10-05 12:18:25.909649: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2022-10-05 12:18:26,837] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-10-05 12:18:26,858] [INFO] Pipeline: ['tok2vec', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'ner']
[2022-10-05 12:18:26,872] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-10-05 12:18:26,874] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-10-05 12:18:27,316] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
INFO:spacy:Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #  

In [ ]:
nlp_ner = spacy.load("/content/model-last")

In [ ]:
import pickle

In [ ]:
with open('model1_pickle_j_extract', 'wb') as f:
  pickle.dump(nlp_ner, f)

In [ ]:
with open("/jtest2.txt") as f1:
  x = f1.read()

In [ ]:
doc1 = nlp_ner(x)

In [ ]:
spacy.displacy.render(doc1, style="ent", jupyter=True)